In [1]:
from src.TFIDF import *
from src.Word2Vector import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xiangzairan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/xiangzairan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/xiangzairan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xiangzairan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/xiangzairan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/xiangzairan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Baseline of Baseline model (Word2Vec)

In [15]:
#class_lis = ['insurance-etc', 'investment', 'medical-sales', 'phising', 'sexual', 'software-sales']

class_lis = ['insurance-etc', 'investment', 'medical-sales', 'software-sales']
#class_lis, vector_size, window, min_count, workers
object_token = w2v_Tokenization(class_lis, 200, 5, 1, 4)

token, word_dic = object_token.get_token_wordDic()
seeds = object_token.modify_seeds()

tokenizing documents...
getting the vector for each word...


In [16]:
object_w2v = Word2vector(token, word_dic, seeds, class_lis)

#get the w2v model accuracy
object_w2v.get_accuracy()

getting the vector for each document...
getting the prediction...
calculating accuracy


(0.22329511164755583, 0.12357727458990425)

## Baseline of Baseline model (TFIDF)
- (TFIDF with the initial seed words): accuracy about 0.5
- (TFIDF with updated seed words): accuracy 0.67

not looking good for baseline model...

In [26]:
class_lis = ['insurance-etc', 'investment', 'medical-sales', 'phising', 'sexual', 'software-sales']

object_token = tfidf_Tokenization(class_lis)
token = object_token.token_X()
seeds = object_token.modify_seeds()

tokenizing documents...


In [27]:
print("currently have", len(token), "of spam email data" )

currently have 2119 of spam email data


In [28]:
object_tfidf = tfidf(token, seeds, class_lis)

#get the tfidf model accuracy
object_tfidf.get_accuracy()

get_idf
getting predictions
calculating accuracy


(0.6715431807456347, 0.6587230004781627)

- With initial seed words: The accuracy scores (0.5441245870693724, 0.5430148863392392) doesn't looks so good for a baseline model, especially for tfidf

- with updated seed words: (0.6715431807456347, 0.6587230004781627)

### Find the common words in each class and update the seed word sets

In [5]:
from collections import defaultdict

def get_common_word(seeds, cla): #input as a list with a single value in it
    
    object_token = tfidf_Tokenization(cla)
    token = object_token.token_X()
    seeds = object_token.modify_seeds()

    object_tfidf = tfidf(token, seeds, cla)
    idf = object_tfidf.get_idf()
    
    #get the frequency ranking for the exsiting seed words: (might remove if its above 100)
    seeds_words = seeds[cla[0]]
    rank = {key: rank for rank, key in enumerate(sorted(idf, key=idf.get, reverse=True), 1)}
    
    seeds_rank_dict = defaultdict(int)
    for s in seeds_words:
        if s in rank:
            seeds_rank_dict[s] = rank[s]
        else:
            seeds_rank_dict[s] = 1000000
    
    #getthe frequency of all the words in this class:
    sort_list = sorted(idf.items(), key=lambda x:x[1], reverse=True)
    
    print("the class", cla, "has ", len(sort_list), "data in it.")
    
    return seeds_rank_dict, sort_list[:100] #return ranking of exsiting seed and the most 100 common words in that class
   
    

In [6]:
print(get_common_word(seeds, ['insurance-etc'])[0])
print(get_common_word(seeds, ['investment'])[0])
print(get_common_word(seeds, ['medical-sales'])[0])
print(get_common_word(seeds, ['phising'])[0])
print(get_common_word(seeds, ['sexual'])[0])
print(get_common_word(seeds, ['software-sales'])[0])

#show ranking (not frequency, larger number means worse)

tokenizing documents...
get_idf
the class ['insurance-etc'] has  6272 data in it.
defaultdict(<class 'int'>, {'credit': 16, 'cash': 40, 'home': 45, 'mortgag': 5, 'rate': 2, 'loan': 19, 'refin': 17})
tokenizing documents...
get_idf
the class ['investment'] has  8351 data in it.
defaultdict(<class 'int'>, {'stock': 14, 'market': 73, 'price': 15, 'invest': 4, 'interest': 39, 'statement': 10, 'secur': 6, 'base': 24, 'risk': 23, 'trade': 22, 'uncertainti': 68})
tokenizing documents...
get_idf
the class ['medical-sales'] has  22355 data in it.
defaultdict(<class 'int'>, {'pill': 13, 'buy': 41, 'medic': 9, 'drug': 11, 'prescript': 8, 'med': 19, 'doctor': 35, 'viagra': 37, 'pain': 52, 'effect': 61})
tokenizing documents...
get_idf
the class ['phising'] has  15046 data in it.
defaultdict(<class 'int'>, {'lotteri': 58, 'win': 68, 'bank': 24, 'award': 84, 'money': 2, 'confidenti': 25, 'agent': 54, 'winner': 78, 'prize': 86, 'unclaim': 93})
tokenizing documents...
get_idf
the class ['sexual'] has 

#### Delete
- insurance_top: insur
- investment_top: profit, earn, money, 
- medical_top: sale, pharmaci, health, enhanc
- phising_top: director, congrat, 
- sexual_top: webcam, fuck
- software_top: sale, buy, order, 

In [7]:
insurance_top = get_common_word(seeds, ['insurance-etc'])[1]
investment_top = get_common_word(seeds, ['investment'])[1]
medical_top = get_common_word(seeds, ['medical-sales'])[1]
phising_top = get_common_word(seeds, ['phising'])[1]
sexual_top = get_common_word(seeds, ['sexual'])[1]
software_top = get_common_word(seeds, ['software-sales'])[1]

tokenizing documents...
get_idf
the class ['insurance-etc'] has  6272 data in it.
tokenizing documents...
get_idf
the class ['investment'] has  8351 data in it.
tokenizing documents...
get_idf
the class ['medical-sales'] has  22355 data in it.
tokenizing documents...
get_idf
the class ['phising'] has  15046 data in it.
tokenizing documents...
get_idf
the class ['sexual'] has  3364 data in it.
tokenizing documents...
get_idf
the class ['software-sales'] has  8282 data in it.


#### Add: frequency in()

- insurance_top: credit (44), cash(30), home(28), 
- investment_top: statement(120), secur(122), base(114), risk(114), trade(114), uncertainti(100)
- medical_top: med(128), doctor(96), viagra(92), pain(79), effect(77), 
- phising_top: money(219), confidenti(138), agent(108), winner(96), prize(92), unclaim(89)
- sexual_top: girl(23), sexual(22), video(20), porn(14), adult(13), cheat(11), hottest(10), teen(10), cum(9) (we have a few data in this category, thats why the even top words' frequency is so low compared to other classes, but the wording is actually very different from the rest of the classes, so we are expecting the model to perform very well for this class)
- software_top: offic(126), system(101), photoshop(82), window(66), offer(77), download(37), server(37)


#### Now (updated seed words)

  "insurance-etc": 
    "credit", "cash", "home", "Mortgage", "rate", "loan", "refinance"
  
  
  "investment": 
    "Stock", "market", "price", "invest", "interest", "statement", "secur", "base", "risk", "trade", "uncertainti"
  
  
  "medical-sales": 
    "Pill", "buy", "medication", "drug", "prescription", "med", "doctor", "viagra", "pain", "effect"
  
  
  "phising": 
    "Lottery", "win", "bank", "award", "money", "confidenti", "agent", "winner", "prize", "unclaim"
  
  
  "sexual": 
    "sex", "horny", "date", "free", "sex", "woman", "girl", "sexual", "video", "porn", "adult", "cheat", "hottest", "teen", "cum" 
  
  
  "software-sales": 
    "Software", "price", "microsoft", "Adobe", "offic", "system", "photoshop", "window", "offer", "download", "server"
  


#### Previous: 
- 'insurance-etc': 'mortgag', 'rate', 'loan', 'refin', 'insur'
- 'investment': 'stock', 'market', 'profit', 'earn', 'money', 'price','invest'
- 'medical-sales': 'pill','sale','buy','pharmaci','health','medic','drug','prescript', 'enhanc'
- 'phising': 'lotteri', 'win', 'director', 'bank', 'award', 'congrat'
- 'sexual': 'webcam', 'sex', 'horni', 'date', 'free', 'sex', 'woman', 'fuck'
- 'software-sales': 'softwar','sale','buy','order','price','microsoft','adob'


Previous seed words frequency ranking in each class:

<img src="./images/previous_seeds_ranking.png" width=900/>

questions:
- does the numbers of seed matters (same for each class? or more seed word for class with less data in it)
- anyway to combine with other model (using the length, subject of the email as well)


TODO:

- try W2V

In [8]:
# dic_scores = defaultdict(int)

# for doc in token:
#     for c in list(seeds.keys()):
#         tfidf = object_tfidf.get_tfidf_stat(doc, seeds[c])
#         dic_scores[c] = tfidf
   